# Steam-Lightweight-GAN

Reference: https://github.com/lucidrains/lightweight-gan

## Check GPU specifications

In [1]:
!nvidia-smi

Tue Dec 29 09:10:46 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install requirements

In [2]:
%pip install google-colab-transfer lightweight-gan moviepy > /dev/null

In [3]:
import colab_transfer as ct

ct.mount_google_drive()

Mounted at /content/drive/


## Load image data

In [4]:
archive_name = 'steam-oneface-small-lr.tar.gz'

In [5]:
# Reference: https://github.com/woctezuma/steam-filtered-image-data#steam-oneface-dataset
archive_dict = {
    'steam-oneface-lr.tar.gz': '1QptHrW9vloTtP--YJsxMY8PZWI2D8NJt',
    'steam-oneface-lr_with_retinaface.tar.gz': '1-0Nk7H6Cn3Nt60EdHG_NWSA8ohi2oBqr',
    'steam-oneface-lr_with_dlib.tar.gz': '1-4RIn9G9Bee2JZ1bK1gkkgkLocHuWJJ4',
    'steam-oneface-small-lr.tar.gz': '1-1V5fDhPo75iDtAbrD18rppV-lf51bPW',
    'steam-oneface-tiny-lr.tar.gz': '1-2sCVgBUmu6LFug1pzBfmL8zNFFBq27F',
}

In [6]:
from pathlib import Path

gdrive_id = archive_dict[archive_name]
print('Processing file {} with id = {}'.format(archive_name, gdrive_id))

if not Path(archive_name).exists():
  !gdown --id {gdrive_id}
  !tar xf {archive_name}

Processing file steam-oneface-small-lr.tar.gz with id = 1-1V5fDhPo75iDtAbrD18rppV-lf51bPW
Downloading...
From: https://drive.google.com/uc?id=1-1V5fDhPo75iDtAbrD18rppV-lf51bPW
To: /content/steam-oneface-small-lr.tar.gz
30.3MB [00:00, 115MB/s] 


## Train

### Parameters

In [7]:
base_name = archive_name.replace('.tar.gz', '')

In [8]:
image_dir = '/content/{}/'.format(base_name)

print('Image directory: {}'.format(image_dir))

Image directory: /content/steam-oneface-small-lr/


In [9]:
run_name = 'run-{}'.format(base_name)

print('Run name: {}'.format(run_name))

Run name: run-steam-oneface-small-lr


In [10]:
model_dir = '/content/drive/MyDrive/lightweight-gan/'
%mkdir -p {model_dir}

### Run

In [ ]:
# An arbitrary augmentation plan:
# -   iter   0k -  27k   [translation]
# -   iter  27k -  54k   [translation,color]
# -   iter  54k - 150k   [translation,color,cutout]

mini_batch_size = 64

!lightweight_gan \
 --data {image_dir} \
 --name {run_name} \
 --models-dir {model_dir} \
 --amp \
 --aug-prob 0.4 \
 --aug-types [translation,color,cutout] \
 --batch-size {mini_batch_size} \
 --gradient-accumulate-every 1 \
 --save-every 1000 \ 


continuing from previous epoch - 62
loading from version 0.14.4
run-steam-oneface-small-lr</content/steam-oneface-small-lr/>:  41% 62000/150000 [00:00<?, ?it/s]G: 3.35 | D: 0.18 | GP: 0.54 | SS: 0.02
run-steam-oneface-small-lr</content/steam-oneface-small-lr/>:  41% 62048/150000 [01:15<38:54:30,  1.59s/it]G: 1.16 | D: 0.28 | GP: 0.41 | SS: 0.02
run-steam-oneface-small-lr</content/steam-oneface-small-lr/>:  41% 62097/150000 [02:34<39:12:56,  1.61s/it]G: 3.52 | D: 0.03 | GP: 0.44 | SS: 0.01
run-steam-oneface-small-lr</content/steam-oneface-small-lr/>:  41% 62104/150000 [02:44<38:11:29,  1.56s/it]

## Sample

In [16]:
!lightweight_gan --generate \
 --name {run_name} \
 --models-dir {model_dir} \


continuing from previous epoch - 62
loading from version 0.14.4
sample images generated at ./results/run-steam-oneface-small-lr/generated-12-29-2020_11-31-56


## Interpolate

In [17]:
!lightweight_gan --generate-interpolation \
 --name {run_name} \
 --models-dir {model_dir} \


continuing from previous epoch - 62
loading from version 0.14.4
100% 100/100 [00:46<00:00,  2.14it/s]
interpolation generated at ./results/run-steam-oneface-small-lr/generated-12-29-2020_11-32-13


In [18]:
import glob

input_path = 'results/{}/'.format(run_name)
gif_files = glob.glob(input_path + '*.gif')

In [19]:
import moviepy.editor as mp

input_name = sorted(gif_files)[-1]
output_name = input_name.replace('.gif', '.mp4')

clip = mp.VideoFileClip(input_name)
clip.write_videofile(output_name)

[MoviePy] >>>> Building video results/run-steam-oneface-small-lr/generated-12-29-2020_11-32-13.mp4
[MoviePy] Writing video results/run-steam-oneface-small-lr/generated-12-29-2020_11-32-13.mp4


100%|█████████▉| 793/794 [02:40<00:00,  4.94it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: results/run-steam-oneface-small-lr/generated-12-29-2020_11-32-13.mp4 



Done.